# Idea Refiner - Flask AI Demo Submission
**Submitted by Prateek Bheevgade**

Idea Refiner is an AI-powered web application that transforms rough, unstructured ideas into detailed, actionable plans instantly. Simply describe your concept in a few words, and our intelligent system analyzes it to generate a comprehensive breakdown including an overview, key features, implementation steps, and potential use cases.

#Key Features
🚀 Instant Transformation - Convert vague thoughts into structured business plans in seconds.

🤖 AI-Powered Analysis - Leverages advanced language models to understand and enhance your ideas

📋 Comprehensive Output - Generates detailed sections covering all aspects of your concept

💾 Easy Export - Download the entire analysis as a clean text file for offline reference

🎨 Modern Interface - Clean, professional UI with smooth animations and responsive design

⚡ Zero Setup - No registration required - just type and transform

#Perfect For
- Entrepreneurs validating startup concepts

- Product managers brainstorming new features

- Students developing project proposals

- Innovators exploring fresh ideas

- Anyone who needs to quickly structure their thoughts

#How It Works?
1. Describe your idea in the input field (e.g., "A smart home automation system")

2. Click Transform to let AI analyze and refine your concept

3. Review the structured breakdown with detailed insights

4. Download the complete analysis as a text file

In [ ]:
import os
os.makedirs('templates', exist_ok=True)
os.makedirs('static/css', exist_ok=True)
os.makedirs('static/js', exist_ok=True)

In [ ]:
%%writefile requirements.txt
Flask==3.0.0
requests==2.31.0


In [ ]:
%%writefile app.py
import os
import requests
from flask import Flask, render_template, request, jsonify, Response
import json

app = Flask(__name__)

# --- CONFIGURATION ---

# 1. API KEY (If your Ollama server requires authentication)
#    User provided key:
API_KEY = "70875543d3ab44c0919cd6cc64107f7f.mpRVNBikG-46m8ncAwoeW9-_"

# 2. OLLAMA URL
#    If you are running Ollama locally, use: "http://localhost:11434/api/generate"
#    If you are using a remote server, paste the URL here.
OLLAMA_API_URL = "http://localhost:11434/api/generate"

# 3. Model
DEFAULT_MODEL = "llama3"

# System Prompt
SYSTEM_PROMPT = """
You are an enthusiastic, encouraging, and expert product manager.
Your goal is to validate the user's idea and make them feel confident, then provide a structured technical breakdown.

**Response Rules:**
1.  **Generate a Title**:
    - Based on the user's idea, create a short, punchy title (3-5 words).
    - **CRITICAL**: The very first line of your response MUST be exactly: `# Title: [Your Title Here]`.
    - Do not bold the 'Title:' label, just `# Title: The Title Text`.

2.  **Start with a Warm, High-Energy Compliment**:
    - Always begin by praising the idea enthusiastically (e.g., "This is a brilliant concept!", "I absolutely love this direction!", "That is a game-changing idea!").
    - Make the user feel like their idea is unique and valuable.
    - **Do NOT** use a header like "Greeting". Just write the paragraph naturally after the title line.

3.  **Transition to the Plan**:
    - After the compliment, immediately dive into the professional details using the headers below.

**Output Format:**

# Title: [3-5 Word Title]

(Enthusiastic greeting paragraph here...)

## 🚀 Overview
(A catchy, clear summary of the idea)

## 🌟 Key Features
(Bulleted list of core components)

## 🔧 Implementation Steps
(High-level technical execution plan)

## 💡 Potential Use Cases
(Who would use this and why)

Keep the mood professional yet very exciting.
"""

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/transform', methods=['POST'])
def transform():
    data = request.json
    user_input = data.get('idea', '')

    if not user_input:
        return jsonify({"error": "Please provide an idea."}), 400

    full_prompt = f"Idea: {user_input}\n\n{SYSTEM_PROMPT}"

    payload = {
        "model": DEFAULT_MODEL,
        "prompt": full_prompt,
        "stream": False,
        "options": {
            "num_predict": 32768
        }
    }

    # Prepare Headers
    headers = {
        "Content-Type": "application/json"
    }
    # Add Authorization header if key is present
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"
        # Some providers might use 'Api-Key' or other headers.
        # For standard OAuth/Proxy, 'Bearer' is safest.
        # If this is ZhipuAI/Other, we might need a specific adapter.

    try:
        response = requests.post(OLLAMA_API_URL, json=payload, headers=headers, timeout=300)

        # Check for HTTP errors
        if response.status_code != 200:
            return jsonify({
                "success": False,
                "error": f"API Error ({response.status_code}): {response.text}"
            }), response.status_code

        result = response.json()

        # Handle different response formats (Ollama vs others)
        if 'response' in result:
            generated_text = result['response']
        elif 'choices' in result and len(result['choices']) > 0:
            # OpenAI/Groq compatible format
            generated_text = result['choices'][0]['message']['content']
        else:
            generated_text = str(result)

        return jsonify({"success": True, "data": generated_text})

    except requests.exceptions.ConnectionError:
        return jsonify({
            "success": False,
            "error": f"Could not connect to: {OLLAMA_API_URL}. Is the server running?"
        }), 503
    except Exception as e:
        return jsonify({"success": False, "error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Vibe Coder | Idea Refiner</title>

    <!-- Fonts -->
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Outfit:wght@300;400;600;700&display=swap" rel="stylesheet">

    <!-- Styles -->
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">

    <!-- Markdown Parser -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <!-- FontAwesome -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
</head>

<body>

    <!-- Developer Profile Trigger -->
    <button id="devProfileBtn" class="dev-profile-btn" title="About Developer">
        <i class="fa-solid fa-code"></i>
    </button>

    <!-- Developer Modal -->
    <div id="devModal" class="dev-modal">
        <div class="dev-modal-content">
            <span class="close-modal">&times;</span>
            <div class="dev-header">
                <div class="dev-avatar">
                    <i class="fa-solid fa-user-tie"></i>
                </div>
                <h2>Prateek Bheevgade</h2>
                <p>AI Developer & Cybersecurity Researcher</p>
            </div>

            <div class="dev-links">
                <a href="mailto:prateek.vijay.bheevgad@gmail.com" class="social-link email" title="Email"
                    target="_blank">
                    <i class="fa-solid fa-envelope"></i>
                </a>
                <a href="https://wa.me/919270823017" class="social-link whatsapp" title="WhatsApp" target="_blank">
                    <i class="fa-brands fa-whatsapp"></i>
                </a>
                <a href="https://www.linkedin.com/in/prateek-bheevgade-67b5573a1" class="social-link linkedin"
                    title="LinkedIn" target="_blank">
                    <i class="fa-brands fa-linkedin-in"></i>
                </a>
                <a href="https://github.com/0xcyberninja-git" class="social-link github" title="GitHub" target="_blank">
                    <i class="fa-brands fa-github"></i>
                </a>
            </div>
        </div>
    </div>

    <div class="container">
        <header>
            <h1>Idea Refiner</h1>
            <p class="subtitle">Transform rough thoughts into structured, demo-ready concepts instantly.</p>
        </header>

        <main>
            <!-- Input Section -->
            <section class="glass-panel input-section">
                <textarea id="ideaInput"
                    placeholder="Describe your idea here... (e.g. 'A mobile app that helps people find hiking buddies based on skill level')"></textarea>

                <div class="action-bar">
                    <div id="loader" class="loading-indicator">
                        <div class="spinner"></div>
                        <span>Processing with AI...</span>
                    </div>
                    <button id="transformBtn" type="button">
                        <span>Transform</span>
                        <svg width="20" height="20" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                            <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2"
                                d="M13 10V3L4 14h7v7l9-11h-7z" />
                        </svg>
                    </button>
                </div>
            </section>

            <!-- Output Section -->
            <section id="outputSection" class="glass-panel output-section">
                <div class="output-header" style="display: flex; justify-content: flex-end; margin-bottom: 1rem;">
                    <button id="downloadBtn" type="button" class="secondary-btn">
                        <span>Download Text</span>
                        <svg width="20" height="20" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                            <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2"
                                d="M4 16v1a3 3 0 003 3h10a3 3 0 003-3v-1m-4-4l-4 4m0 0l-4-4m4 4V4" />
                        </svg>
                    </button>
                </div>
                <div id="resultContent" class="result-content">
                    <!-- AI Content will be injected here -->
                </div>
            </section>
        </main>
    </div>

    <!-- Script -->
    <script src="{{ url_for('static', filename='js/script.js') }}"></script>
</body>

</html>

In [ ]:
%%writefile static/css/style.css
:root {
    --bg-color: #f3f4f6;
    --card-bg: #ffffff;
    --primary-color: #1e3a8a;
    /* Deep Royal Blue */
    --accent-color: #ea580c;
    /* Burned Orange */
    --text-primary: #1f2937;
    --text-secondary: #6b7280;
    --border-color: #e5e7eb;
    --shadow-soft: 0 4px 6px -1px rgba(0, 0, 0, 0.05), 0 2px 4px -1px rgba(0, 0, 0, 0.03);
    --shadow-hover: 0 10px 15px -3px rgba(0, 0, 0, 0.08), 0 4px 6px -2px rgba(0, 0, 0, 0.04);
    --font-heading: 'Outfit', sans-serif;
    --font-body: 'Outfit', sans-serif;
}

* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: var(--font-body);
    background-color: var(--bg-color);
    color: var(--text-primary);
    min-height: 100vh;
    display: flex;
    flex-direction: column;
    align-items: center;
    padding-top: 4rem;
    padding-bottom: 4rem;
}

.container {
    width: 100%;
    max-width: 960px;
    padding: 0 2rem;
}

/* Header */
header {
    text-align: center;
    margin-bottom: 3.5rem;
}

h1 {
    font-family: var(--font-heading);
    font-size: 3.5rem;
    font-weight: 800;
    color: var(--primary-color);
    margin-bottom: 0.75rem;
    letter-spacing: -0.03em;
    line-height: 1.1;
    text-transform: none;
    /* Removed uppercase */
}

p.subtitle {
    color: var(--text-secondary);
    font-size: 1.25rem;
    font-weight: 400;
    max-width: 550px;
    margin: 0 auto;
    line-height: 1.6;
}

/* Card Style (Clean SaaS) */
.glass-panel {
    background: var(--card-bg);
    border: 1px solid var(--border-color);
    border-radius: 16px;
    /* Smooth rounded corners */
    padding: 2.5rem;
    box-shadow: var(--shadow-soft);
    transition: all 0.3s ease;
}

.glass-panel:hover {
    box-shadow: var(--shadow-hover);
    transform: translateY(-2px);
}

/* Input Area */
.input-section {
    position: relative;
    margin-bottom: 2rem;
}

textarea {
    width: 100%;
    min-height: 180px;
    background: #f9fafb;
    border: 1px solid var(--border-color);
    border-radius: 12px;
    padding: 1.5rem;
    color: var(--text-primary);
    font-family: var(--font-body);
    font-size: 1.1rem;
    font-weight: 400;
    resize: vertical;
    transition: all 0.2s ease;
    line-height: 1.6;
}

textarea:focus {
    outline: none;
    background: #fff;
    border-color: var(--primary-color);
    box-shadow: 0 0 0 3px rgba(30, 58, 138, 0.1);
    transform: none;
}

.action-bar {
    display: flex;
    justify-content: flex-end;
    align-items: center;
    margin-top: 1.5rem;
    gap: 1.5rem;
}

button {
    background-color: var(--primary-color);
    border: 1px solid var(--primary-color);
    border-radius: 8px;
    /* Softer button */
    padding: 12px 32px;
    color: #fff;
    font-weight: 600;
    font-size: 1.05rem;
    cursor: pointer;
    transition: all 0.2s ease;
    display: flex;
    align-items: center;
    gap: 10px;
    letter-spacing: 0.02em;
    text-transform: none;
    /* Normal case */
}

button:hover {
    background-color: #172554;
    /* Darker Blue */
    color: #fff;
    transform: translateY(-1px);
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
}

button:disabled {
    background-color: #94a3b8;
    border-color: #94a3b8;
    cursor: not-allowed;
    transform: none;
    box-shadow: none;
}

button svg {
    width: 20px;
    height: 20px;
}

/* Secondary Button (Download) */
.secondary-btn {
    background-color: #fff;
    color: var(--text-secondary);
    border: 1px solid var(--border-color);
    font-weight: 500;
}

.secondary-btn:hover {
    background-color: #f8fafc;
    color: var(--primary-color);
    border-color: var(--primary-color);
    box-shadow: var(--shadow-soft);
}

/* --- Developer Profile Feature --- */

/* Floating Trigger Button */
.dev-profile-btn {
    position: fixed;
    top: 25px;
    right: 25px;
    width: 52px;
    height: 52px;
    background-color: var(--primary-color);
    border: none;
    border-radius: 12px;
    color: #ffffff;
    display: flex;
    justify-content: center;
    align-items: center;
    z-index: 99999;
    cursor: pointer;
    box-shadow: 0 4px 12px rgba(30, 58, 138, 0.25);
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
}

.dev-profile-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 10px 15px -3px rgba(30, 58, 138, 0.3);
    background-color: #172554;
    color: #fff;
}

.dev-profile-btn i {
    font-size: 1.3rem;
}

/* Modal Overlay */
.dev-modal {
    display: none;
    position: fixed;
    z-index: 2000;
    left: 0;
    top: 0;
    width: 100%;
    height: 100%;
    background-color: rgba(31, 41, 55, 0.3);
    /* Dimmed cool gray */
    backdrop-filter: blur(5px);
    animation: fadeIn 0.3s;
}

.dev-modal-content {
    background-color: #fff;
    margin: 15% auto;
    padding: 3rem;
    border-radius: 24px;
    border: 1px solid var(--border-color);
    width: 90%;
    max-width: 440px;
    box-shadow: 0 25px 50px -12px rgba(0, 0, 0, 0.15);
    text-align: center;
    position: relative;
    animation: slideDown 0.4s;
}

.close-modal {
    position: absolute;
    top: 20px;
    right: 25px;
    color: #9ca3af;
    font-size: 24px;
    cursor: pointer;
    transition: color 0.2s;
}

.close-modal:hover {
    color: var(--text-primary);
}

.dev-avatar {
    width: 88px;
    height: 88px;
    background: #eff6ff;
    border: none;
    border-radius: 50%;
    margin: 0 auto 1.5rem;
    display: flex;
    align-items: center;
    justify-content: center;
    color: var(--primary-color);
    font-size: 2.2rem;
}

.dev-header h2 {
    font-size: 1.75rem;
    margin-bottom: 0.5rem;
    color: var(--text-primary);
    letter-spacing: -0.02em;
}

.dev-header p {
    color: var(--text-secondary);
    margin-bottom: 2.5rem;
    font-weight: 400;
    font-size: 1rem;
}

.dev-links {
    display: flex;
    justify-content: center;
    gap: 1rem;
}

.social-link {
    width: 48px;
    height: 48px;
    border-radius: 12px;
    background: #fff;
    border: 1px solid var(--border-color);
    color: var(--text-secondary);
    display: flex;
    align-items: center;
    justify-content: center;
    font-size: 1.25rem;
    transition: all 0.2s ease;
    text-decoration: none;
}

.social-link:hover {
    transform: translateY(-2px);
    border-color: var(--primary-color);
    color: var(--primary-color);
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.05);
}

.social-link.whatsapp:hover {
    color: #25D366;
    border-color: #25D366;
}

.social-link.linkedin:hover {
    color: #0077b5;
    border-color: #0077b5;
}

.social-link.github:hover {
    color: #333;
    border-color: #333;
}

.social-link.email:hover {
    color: #EA4335;
    border-color: #EA4335;
}


/* Output Area */
.output-section {
    display: none;
}

.output-section.active {
    display: block;
    animation: slideUp 0.5s ease-out;
}

/* Collapsible Sections (Clean Accordion) */
.result-section {
    margin-bottom: 1rem;
    border: 1px solid var(--border-color);
    border-radius: 12px;
    background: #fff;
    transition: all 0.2s ease;
    overflow: hidden;
}

.result-section.expanded {
    border-color: #bfdbfe;
    /* Soft Blue */
    box-shadow: 0 4px 6px -1px rgba(59, 130, 246, 0.05);
}

.section-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 1.25rem 1.75rem;
    background: #fff;
    cursor: pointer;
    border-bottom: 1px solid transparent;
}

.result-section.expanded .section-header {
    background: #f8fafc;
    border-bottom: 1px solid #f1f5f9;
}

.section-header:hover {
    background: #f8fafc;
}

.section-header h2 {
    margin: 0 !important;
    border: none !important;
    font-size: 1.25rem !important;
    font-weight: 600 !important;
    color: var(--text-primary);
}

.result-section.expanded .section-header h2 {
    color: var(--primary-color);
}

.toggle-icon {
    font-size: 1.5rem;
    color: #9ca3af;
    font-weight: 300;
    transition: transform 0.3s ease;
}

.result-section.expanded .toggle-icon {
    transform: rotate(180deg);
    color: var(--primary-color);
}

.section-content {
    display: none;
    padding: 1.75rem;
    color: #374151;
    line-height: 1.7;
}

.result-section.expanded .section-content {
    display: block;
    animation: fadeIn 0.4s ease;
}

/* Intro Section */
.intro-section {
    background: #fff;
    border: 1px solid #fed7aa;
    /* Soft Orange */
    border-radius: 12px;
    padding: 1.5rem;
    margin-bottom: 2rem;
    box-shadow: 0 4px 6px -1px rgba(255, 237, 213, 0.4);
}

.intro-section p {
    font-size: 1.1rem;
    font-weight: 500;
    color: #9a3412;
    /* Dark Orange */
    margin: 0;
    font-style: normal;
}

/* Result Content Typography */
.result-content p {
    margin-bottom: 1.2rem;
}

.result-content ul {
    list-style-type: none;
    padding-left: 0.5rem;
}

.result-content li {
    position: relative;
    padding-left: 1.5rem;
    margin-bottom: 0.8rem;
    font-weight: 400;
}

.result-content li::before {
    content: "•";
    position: absolute;
    left: 0;
    color: var(--accent-color);
    font-size: 1.2rem;
    top: -0.2rem;
}

/* Loading State */
.loading-indicator {
    display: none;
    align-items: center;
    gap: 12px;
    font-size: 0.95rem;
    font-weight: 500;
    color: var(--text-secondary);
}

.spinner {
    width: 20px;
    height: 20px;
    border: 2px solid #e5e7eb;
    border-top-color: var(--primary-color);
    border-radius: 50%;
    animation: spin 0.8s linear infinite;
}

/* Animations */
@keyframes slideUp {
    from {
        opacity: 0;
        transform: translateY(20px);
    }

    to {
        opacity: 1;
        transform: translateY(0);
    }
}

@keyframes spin {
    to {
        transform: rotate(360deg);
    }
}

@keyframes fadeIn {
    from {
        opacity: 0;
    }

    to {
        opacity: 1;
    }
}

@keyframes slideDown {
    from {
        transform: translateY(-20px);
        opacity: 0;
    }

    to {
        transform: translateY(0);
        opacity: 1;
    }

    /* PDF Cover Page Theme (Creative / Idea ReFiner Style) */
    .pdf-cover {
        position: relative;
        width: 100%;
        height: 1100px;
        background: #fdfbff;
        /* Very light tint */
        background-image: radial-gradient(#e5e7eb 1px, transparent 1px);
        background-size: 20px 20px;
        padding: 60px;
        display: flex;
        flex-direction: column;
        align-items: center;
        text-align: center;
        font-family: 'Outfit', sans-serif;
        color: #1a1a1a;
        overflow: hidden;
    }

    /* Creative Fluid Shapes */
    .corner-graphic {
        position: absolute;
        top: -50px;
        left: -50px;
        width: 400px;
        height: 400px;
        background: linear-gradient(135deg, #FF6B6B 0%, #FFE66D 100%);
        border-radius: 60% 40% 30% 70% / 60% 30% 70% 40%;
        opacity: 0.9;
        z-index: 10;
        box-shadow: 10px 10px 60px rgba(0, 0, 0, 0.1);
    }

    .corner-strip {
        position: absolute;
        top: 50px;
        left: 100px;
        width: 200px;
        height: 200px;
        background: rgba(255, 107, 107, 0.2);
        border-radius: 30% 70% 70% 30% / 30% 30% 70% 70%;
        z-index: 5;
        animation: spin 20s linear infinite;
    }

    /* Bottom Right Graphic */
    .corner-graphic-bottom {
        position: absolute;
        bottom: -80px;
        right: -80px;
        width: 350px;
        height: 350px;
        background: linear-gradient(135deg, #4ECDC4 0%, #556270 100%);
        border-radius: 40% 60% 70% 30% / 40% 50% 60% 50%;
        z-index: 10;
        opacity: 0.9;
    }

    .cover-header {
        margin-top: 150px;
        /* More space for the blob */
        margin-bottom: 60px;
        z-index: 20;
    }

    .logo-circle {
        width: 110px;
        height: 110px;
        background: #fff;
        border: none;
        border-radius: 30% 70% 70% 30% / 30% 30% 70% 70%;
        margin: 0 auto 20px;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 3rem;
        color: #FF6B6B;
        box-shadow: 0 10px 25px rgba(0, 0, 0, 0.1);
    }

    .cover-header h1 {
        font-size: 3.5rem;
        font-weight: 800;
        color: #2c3e50;
        margin-bottom: 5px;
        text-transform: none;
        letter-spacing: -1px;
    }

    .cover-header .role {
        font-size: 1.2rem;
        color: #FF6B6B;
        /* Accent color text */
        font-weight: 600;
        margin-bottom: 10px;
        letter-spacing: 0.5px;
        text-transform: uppercase;
    }

    .cover-header .contact {
        font-size: 0.95rem;
        color: #7f8c8d;
        font-family: monospace;
    }

    /* Title Section */
    .cover-main {
        margin: 40px 0;
        z-index: 20;
        position: relative;
        padding: 30px;
    }

    .cover-main::before {
        content: '"';
        position: absolute;
        top: -40px;
        left: 50%;
        transform: translateX(-50%);
        font-size: 8rem;
        color: rgba(200, 200, 200, 0.2);
        font-family: serif;
    }

    .cover-main h1.report-title {
        font-size: 3.5rem;
        font-weight: 900;
        background: -webkit-linear-gradient(45deg, #FF6B6B, #556270);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        line-height: 1.1;
        margin-bottom: 0;
    }

    .cover-footer {
        margin-top: auto;
        margin-bottom: 60px;
        font-size: 0.9rem;
        color: #95a5a6;
        z-index: 20;
        border-top: 1px dashed #BDC3C7;
        padding-top: 20px;
        width: 60%;
    }

In [ ]:
%%writefile static/js/script.js
document.addEventListener('DOMContentLoaded', () => {
    const transformBtn = document.getElementById('transformBtn');
    const ideaInput = document.getElementById('ideaInput');
    const outputSection = document.getElementById('outputSection');
    const resultContent = document.getElementById('resultContent');
    const loader = document.getElementById('loader');
    const downloadBtn = document.getElementById('downloadBtn');

    // Variables to store data for PDF
    let currentTitle = "Idea Transformation";
    let currentMarkdown = "";

    transformBtn.addEventListener('click', async () => {
        const idea = ideaInput.value.trim();
        if (!idea) {
            ideaInput.focus();
            ideaInput.style.borderColor = '#ef4444';
            setTimeout(() => ideaInput.style.borderColor = '', 2000);
            return;
        }

        // UI State: Loading
        setLoading(true);
        outputSection.classList.remove('active');

        try {
            const response = await fetch('/transform', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ idea })
            });

            const data = await response.json();

            if (data.success) {
                let rawText = data.data;

                // --- Parse Title (Generated by AI) ---
                // Regex to handle '# Title:', 'Title:', '**Title:**', etc.
                const titleRegex = /^(?:#\s*|\*\*\s*)?Title:?\s*(.*?)(\n|$)/i;
                const match = rawText.match(titleRegex);

                if (match && match[1]) {
                    currentTitle = match[1].trim();
                    // Remove the Title line from the text shown to user
                    rawText = rawText.replace(match[0], '').trim();
                } else {
                    currentTitle = "Strategic Implementation Plan";
                }

                currentMarkdown = rawText; // Save for PDF

                // Parse Markdown
                const htmlContent = marked.parse(rawText);

                resultContent.innerHTML = htmlContent;

                // Show Result
                outputSection.classList.add('active');

                // Smooth scroll to results
                outputSection.scrollIntoView({ behavior: 'smooth', block: 'start' });
            } else {
                showError(data.error || 'Something went wrong.');
            }
        } catch (error) {
            console.error('Error:', error);
            showError('Failed to connect to the server.');
        } finally {
            setLoading(false);
        }
    });

    // ===== TEXT DOWNLOAD HANDLER =====
    downloadBtn.addEventListener('click', () => {
        const originalText = downloadBtn.innerHTML;
        downloadBtn.innerHTML = '<span>Downloading...</span>';

        try {
            // Create header with title and metadata
            const dateStr = new Date().toLocaleDateString('en-US', {
                year: 'numeric',
                month: 'long',
                day: 'numeric'
            });

            const textContent = `=====================================================
IDEA REFINER - AI-POWERED ANALYSIS
=====================================================

Title: ${currentTitle}
Generated: ${dateStr}
Developed by: Prateek Bheevgade

=====================================================

${currentMarkdown}

=====================================================
Generated by AI Idea Refiner
Developed by Prateek Bheevgade
=====================================================`;

            // Create blob and download
            const blob = new Blob([textContent], { type: 'text/plain;charset=utf-8' });
            const url = URL.createObjectURL(blob);

            // Sanitize filename
            const cleanTitle = currentTitle
                .replace(/[^a-z0-9\s]/gi, '')
                .trim()
                .replace(/\s+/g, '_')
                .toLowerCase();
            const filename = `${cleanTitle || 'idea_export'}.txt`;

            // Create and trigger download link
            const downloadLink = document.createElement('a');
            downloadLink.href = url;
            downloadLink.download = filename;
            document.body.appendChild(downloadLink);
            downloadLink.click();

            // Cleanup
            setTimeout(() => {
                document.body.removeChild(downloadLink);
                URL.revokeObjectURL(url);
            }, 100);

            console.log(`✓ Downloaded: ${filename}`);

        } catch (error) {
            console.error('Download failed:', error);
            alert('Failed to download file. Please try again.');
        } finally {
            downloadBtn.innerHTML = originalText;
        }
    });

    // --- Developer Profile Modal Logic ---
    const devModal = document.getElementById('devModal');
    const devBtn = document.getElementById('devProfileBtn');
    const closeSpan = document.getElementsByClassName('close-modal')[0];

    // Open Modal
    devBtn.onclick = function () {
        devModal.style.display = 'block';
    }

    // Close on X
    closeSpan.onclick = function () {
        devModal.style.display = 'none';
    }

    // Close on Outside Click
    window.onclick = function (event) {
        if (event.target == devModal) {
            devModal.style.display = 'none';
        }
    }

    function setLoading(isLoading) {
        transformBtn.disabled = isLoading;
        if (isLoading) {
            loader.style.display = 'flex';
            transformBtn.innerHTML = 'Thinking...';
        } else {
            loader.style.display = 'none';
            transformBtn.innerHTML = `
                <span>Transform</span>
                <svg width="20" height="20" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                    <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M13 10V3L4 14h7v7l9-11h-7z" />
                </svg>
            `;
        }
    }

    function showError(msg) {
        resultContent.innerHTML = `<div style="color: #ef4444; padding: 1rem; border: 1px solid #ef4444; border-radius: 8px; background: rgba(239, 68, 68, 0.1);">Error: ${msg}</div>`;
        outputSection.classList.add('active');
    }
});


In [ ]:
# Install dependencies (uncomment if needed)
# !pip install -r requirements.txt

# Run the app
!python app.py

 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
 * Debugger is active!
 * Debugger PIN: 462-506-135
